# Задание по программированию: 1NN против RandomForest

Курс "Обучение на размеченных данных" https://www.coursera.org/learn/supervised-learning/home/welcome

Автор: Сметанин Александр

Дата: 09.06.2018

## Введение

В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

In [2]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [289]:
# Write answers into a file
def write_answer(filename, results):
    with open(filename, "w") as fout:
        fout.write(" ".join([str(res) for res in results]))

In [3]:
ds_digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(ds_digits.data, 
                                                    ds_digits.target, 
                                                    test_size=0.25, 
                                                    random_state=3634, 
                                                    shuffle = False)


Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

## Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [146]:
# Функция, вычисляющая евклидово расстояние между точками. Параметры a и b - числовые векторы равной длины
def euc_dist(a, b):
    try: 
        return np.sqrt(sum(map(lambda x,y: (x-y)**2, a, b)))
    except TypeError:
        print 'Euc_dist(): TypeError!'
        return None

In [238]:
# Функция, вычисляющая расстояние от точки point до всех точек массива matrix при помощи фукции fun.
# Размерности point и элементов matrix должны совпадать.
# Результат: список пар (расстояние, класс = target)
def dist_list(point, data, target, fun):
    res = list()
    for dat, tgt in zip(data, target):
        d = fun(point, dat)
        res.append([d, tgt])
    return res

In [249]:
# Функция, присваивающая класс объекту методом 1NN.
# test_point - объект
# train_data, train_target - матрица обучающей выборки, обучающий вектор классов
# fun - функция, вычисляющая расстояние между точками
# Результат: класс объекта
def NN1(test_point, train_data, train_target, fun):
    dists = dist_list(test_point, train_data, train_target, fun)
    dists.sort()
    return dists[0][1]

In [260]:
%%time
# Программа
predict = list()
for obj in X_test:
    predict.append(NN1(obj, X_train, y_train, euc_dist))

Wall time: 36 s


Время работы при использовании сортировки списков = 36 сек.

In [293]:
# Вычисляем долю ошибок и записываем ее в файл
fault_rate = 1 - metrics.accuracy_score(y_test, predict)
print fault_rate
write_answer('Week_5_Task_3_Answer_1.txt', [fault_rate])

0.0377777777777778


In [300]:
# Покажем метрики качества
print 'Accuracy =', metrics.accuracy_score(y_test, predict)
print 'Precision =', metrics.precision_score(y_test, predict, average='macro')
print 'Recall =', metrics.recall_score(y_test, predict, average='macro')
print 'F-metrics =', metrics.f1_score(y_test, predict, average='macro')
print '\nConfusion matrix =\n', metrics.confusion_matrix(y_test, predict)
print '\nClassification report\n', metrics.classification_report(y_test, predict)

Accuracy = 0.9622222222222222
Precision = 0.9631395159287652
Recall = 0.962339056103327
F-metrics = 0.9620276013344361

Confusion matrix =
[[43  0  0  0  0  0  0  0  0  0]
 [ 0 46  0  0  0  0  0  0  0  0]
 [ 0  0 43  0  0  0  0  0  0  0]
 [ 0  0  0 41  0  2  0  1  2  1]
 [ 0  0  0  0 45  0  0  0  0  3]
 [ 0  0  0  0  0 44  1  0  0  0]
 [ 0  0  0  0  0  0 47  0  0  0]
 [ 0  0  0  0  0  0  0 45  0  0]
 [ 0  3  0  0  0  0  0  0 37  1]
 [ 0  0  0  1  0  2  0  0  0 42]]

Classification report
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        43
          1       0.94      1.00      0.97        46
          2       1.00      1.00      1.00        43
          3       0.98      0.87      0.92        47
          4       1.00      0.94      0.97        48
          5       0.92      0.98      0.95        45
          6       0.98      1.00      0.99        47
          7       0.98      1.00      0.99        45
          8       0.95      0

In [296]:
print metrics.classification_report(y_test, predict)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        43
          1       0.94      1.00      0.97        46
          2       1.00      1.00      1.00        43
          3       0.98      0.87      0.92        47
          4       1.00      0.94      0.97        48
          5       0.92      0.98      0.95        45
          6       0.98      1.00      0.99        47
          7       0.98      1.00      0.99        45
          8       0.95      0.90      0.92        41
          9       0.89      0.93      0.91        45

avg / total       0.96      0.96      0.96       450



## Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [303]:
%%time
RF_classifier = RandomForestClassifier(n_estimators=1000, random_state=3634)
RF_classifier.fit(X_train, y_train)
RF_predict = RF_classifier.predict(X_test)

Wall time: 4.46 s


Время работы классификатора RandomForest = 4.46 сек.

In [304]:
# Вычисляем долю ошибок и записываем ее в файл
fault_rate = 1 - metrics.accuracy_score(y_test, RF_predict)
print fault_rate
write_answer('Week_5_Task_3_Answer_2.txt', [fault_rate])

0.06444444444444442


In [305]:
# Покажем метрики качества
print 'Accuracy =', metrics.accuracy_score(y_test, RF_predict)
print 'Precision =', metrics.precision_score(y_test, RF_predict, average='macro')
print 'Recall =', metrics.recall_score(y_test, RF_predict, average='macro')
print 'F-metrics =', metrics.f1_score(y_test, RF_predict, average='macro')
print '\nConfusion matrix =\n', metrics.confusion_matrix(y_test, RF_predict)
print '\nClassification report\n', metrics.classification_report(y_test, RF_predict)

Accuracy = 0.9355555555555556
Precision = 0.9376222308262274
Recall = 0.9356498075293663
F-metrics = 0.9344426879176548

Confusion matrix =
[[42  0  0  0  1  0  0  0  0  0]
 [ 0 43  0  0  0  0  0  0  0  3]
 [ 1  0 42  0  0  0  0  0  0  0]
 [ 0  0  0 36  0  2  0  2  7  0]
 [ 0  0  0  0 45  0  0  3  0  0]
 [ 0  0  0  0  0 44  1  0  0  0]
 [ 0  0  0  0  0  0 47  0  0  0]
 [ 0  0  0  0  0  0  0 45  0  0]
 [ 0  2  0  0  1  1  0  1 35  1]
 [ 0  0  0  0  0  2  0  0  1 42]]

Classification report
             precision    recall  f1-score   support

          0       0.98      0.98      0.98        43
          1       0.96      0.93      0.95        46
          2       1.00      0.98      0.99        43
          3       1.00      0.77      0.87        47
          4       0.96      0.94      0.95        48
          5       0.90      0.98      0.94        45
          6       0.98      1.00      0.99        47
          7       0.88      1.00      0.94        45
          8       0.81      

## Выводы

На выбранном наборе данных точность выше у метода 1NN, а скорость работы - у классификатора RandomForest.

In [240]:
index = range(30, 51)
index2 = range(30,49)
p = X_test[0,:10]
m = X_train[index,:10]
t = y_train[index2]

In [241]:
val = nearest(p,m,t,euc_dist)
val

[[13.228756555322953, 0],
 [17.146428199482248, 9],
 [13.784048752090222, 5],
 [18.920887928424502, 5],
 [17.832554500127006, 6],
 [16.76305461424021, 5],
 [13.601470508735444, 0],
 [22.759613353482084, 9],
 [10.488088481701515, 8],
 [10.583005244258363, 9],
 [12.96148139681572, 8],
 [16.3707055437449, 4],
 [20.049937655763422, 1],
 [10.14889156509222, 7],
 [5.830951894845301, 7],
 [8.774964387392123, 3],
 [25.787593916455254, 5],
 [18.0, 1],
 [12.489995996796797, 0]]

In [242]:
val.sort()

In [243]:
val

[[5.830951894845301, 7],
 [8.774964387392123, 3],
 [10.14889156509222, 7],
 [10.488088481701515, 8],
 [10.583005244258363, 9],
 [12.489995996796797, 0],
 [12.96148139681572, 8],
 [13.228756555322953, 0],
 [13.601470508735444, 0],
 [13.784048752090222, 5],
 [16.3707055437449, 4],
 [16.76305461424021, 5],
 [17.146428199482248, 9],
 [17.832554500127006, 6],
 [18.0, 1],
 [18.920887928424502, 5],
 [20.049937655763422, 1],
 [22.759613353482084, 9],
 [25.787593916455254, 5]]

In [244]:
val[0][1]

7

In [248]:
NN1(p,m,t,euc_dist)

7